Export txt into csv

In [2]:
import csv

# Read data from the text file
with open('dynamic_B1_A2_0.txt', 'r') as file:
    lines = file.readlines()

# Process data
data = []
for line in lines:
    if line.strip() and not line.startswith('*'):
        parts = line.strip().split()
        tracker = parts[0].strip(':')
        id = parts[1]
        x = parts[2]
        y = parts[3]
        z = parts[4]
        timestamp = parts[5]
        data.append([tracker, id, x, y, z, timestamp])

# Write data to a CSV file
with open('output.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Tracker', 'ID', 'X', 'Y', 'Z', 'Timestamp'])
    csvwriter.writerows(data)

print("Data exported in CSV")


Data has been successfully written to output.csv


Librairies

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
plt.switch_backend('Qt5Agg')

Loading data and cleaning 

In [3]:
data = pd.read_csv("output.csv")
data_n = data.drop(columns=["Tracker", "ID"])

Plot the raw Data in animation

In [10]:
# Initialize the plot
fig, ax = plt.subplots()
line, = ax.plot([], [], 'ro-')

# Set plot limits
ax.set_xlim(min(data_n['X']) - 1, max(data_n['X']) + 1)
ax.set_ylim(min(data_n['Y']) - 1, max(data_n['Y']) + 1)

# Initialization function
def init():
    line.set_data([], [])
    return line,

# Animation function
def animate(i):
    x = data_n['X'][:i]
    y = data_n['Y'][:i]
    line.set_data(x, y)
    return line,

# Create the animation
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=data_n.shape[0], interval=200, blit=True)

# Show the plot with animation
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Raw Data Plot')
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [15]:
# Define the EMA function
def exponential_moving_average(data, alpha=0.2):
    ema = [data[0]]
    for value in data[1:]:
        ema.append(alpha * value + (1 - alpha) * ema[-1])
    return ema

# Calculate EMA for the 'X' and 'Y' columns
data_n['EMA_X'] = exponential_moving_average(data_n['X'])
data_n['EMA_Y'] = exponential_moving_average(data_n['Y'])

# Plot real-time data and EMA
plt.figure(figsize=(14, 7))
plt.plot(data_n["Timestamp"], data_n["X"], 'r-', label='X')
plt.plot(data_n["Timestamp"], data_n["EMA_X"], 'b-', label='EMA_X')
plt.plot(data_n["Timestamp"], data_n["Y"], 'g-', label='Y')
plt.plot(data_n["Timestamp"], data_n["EMA_Y"], 'm-', label='EMA_Y')

plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.legend()
plt.title('EMA')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()